# Model Development

In [1]:
%load_ext autoreload
%autoreload 2
p = print

from os.path import join
import pickle

import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, Lasso, MultiTaskLassoCV, Ridge, RidgeCV, MultiTaskElasticNet, MultiTaskElasticNetCV, ElasticNet, ElasticNetCV
from sklearn.metrics import mean_absolute_error, mean_squared_error

from crypr.util import get_project_path

In [2]:
"""
Import Data.
"""
SYM = 'BTC'
Ty = 1
Tx = 72
MAX_LAG = 72
wavelet = 'haar_smooth'
data_dir = join(get_project_path(), 'data', 'processed')
models_dir = join(get_project_path(), 'models')

X_train = np.load(join(data_dir, 'X_train_{}.npy'.format(SYM)))
Y_train = np.load(join(data_dir, 'y_train_{}.npy'.format(SYM)))
X_test = np.load(join(data_dir, 'X_test_{}.npy'.format(SYM)))
Y_test = np.load(join(data_dir, 'y_test_{}.npy'.format(SYM)))

N_FEATURES = X_train.shape[2]

p(X_train.shape)
X_train = X_train.reshape((-1, Tx*N_FEATURES))
X_test = X_test.reshape((-1, Tx*N_FEATURES))
p(X_train.shape)

(5586, 72, 18)
(5586, 1296)


# Let's try a simple linear regression model

In [3]:
lr_model = LinearRegression()
lr_model = lr_model.fit(X_train, Y_train)
lr_predict = lr_model.predict(X_test)

p(mean_absolute_error(y_pred=lr_predict, y_true=Y_test))
p(mean_squared_error(y_pred=lr_predict, y_true=Y_test))

0.5428762980033514
0.994009168671955


In [4]:
# Save model
# with open(join(models_dir, 'linear_model_{}.pkl'.format(SYM)), 'wb') as output_file:
#     s = pickle.dump(lr_model, output_file)

# Other Linear Regression

## Lasso

In [5]:
lasso_params = {
    'alpha': [0.01],
}

lasso_model = Lasso(alpha=0.01, max_iter=10000)
lasso_model = lasso_model.fit(X=X_train, y=Y_train)

lasso_predict = lasso_model.predict(X_test)

p(mean_absolute_error(lasso_predict, Y_test))
p(mean_squared_error(lasso_predict, Y_test))

0.485266489188477
0.9116682864063553


/Users/daniel_stevenson/.conda/envs/crypto_predict/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


## Ridge


In [6]:
ridge_model = Ridge(alpha=0.01)
ridge_model = ridge_model.fit(X=X_train, y=Y_train)

ridge_predict = ridge_model.predict(X_test)

p(mean_absolute_error(ridge_predict, Y_test))
p(mean_squared_error(ridge_predict, Y_test))

0.5428389780118628
0.9939320679292416


## Elastic Net

In [7]:
enet_params = {
    'alpha': [1e-7],
}

enet_model = MultiTaskElasticNetCV(alphas=enet_params['alpha'])
enet_model = enet_model.fit(X=X_train, y=Y_train)

enet_predict = enet_model.predict(X_test)

p(mean_absolute_error(enet_predict, Y_test))
p(mean_squared_error(enet_predict, Y_test))

/Users/daniel_stevenson/.conda/envs/crypto_predict/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/daniel_stevenson/.conda/envs/crypto_predict/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/daniel_stevenson/.conda/envs/crypto_predict/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/daniel_stevenson/.conda/envs/crypto_predict/lib/py

0.48173897411556726
0.9216886342541981


/Users/daniel_stevenson/.conda/envs/crypto_predict/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:1822: ConvergenceWarning: Objective did not converge, you might want to increase the number of iterations
  ConvergenceWarning)
